# Python code to control USRP X310.

From https://pysdr.org/content/usrp.html

To increase buffer size and maximum ethernet maximum transmit unit, run:

```
sudo sysctl -w net.core.wmem_max=24862979
sudo ip link set dev enp47s0f0 mtu 9000  # or for whatever ethernet card
sudo ip link set dev enp47s0f1 mtu 9000
```

# Spectrum and Recording: "Empowered Worker Threads"

* Lock the lock.
* Look to see what others are doing -- either taking spectra or recording data
* If Nthreads-1 others are recording data, I had better take spectra
* Otherwise, mask out areas of the spectrum where others are taking data and look for the highest peak above a threshold.
* If it's worth taking data there, do that.
* Otherwise, pick a 1/Nthreads piece of spectra and take that.
* Unlock the lock

What data do we need?

* List of what's currently being done:
   - ('record', center_freq)
   - ('spectrum', chunk_number)
* Current spectra, along with how old each little NFFT-long segment is maybe so we can update the oldest spectral bins first

If we are taking spectra and find a loud signal, should we instantly bail out and start recording? Yes, probably. And without gaps.

If we are recording, when should we stop? After a minimal timeout or if the algorithm above says that something else is more interesting (record elsewhere or go back to taking spectra)


In [ ]:
%pylab inline

In [ ]:
import time, datetime
import threading
import uhd
import matplotlib.mlab  # for psd

In [ ]:
#threading.enumerate()

In [ ]:
# Load the Bokeh stuff for jupyter notebooks
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import bokeh.palettes
output_notebook()

In [ ]:
# set up a dummy plot to be updated
source = ColumnDataSource(
    data={
        "freqs_MHz":  [0],  # x
        "powers_dB":  [0],  # y
    }
)
source_marker = ColumnDataSource(
    data={
        "marker_MHz": [], # x
        "marker_dB":  [],  # y
    }
)

In [ ]:
p = figure(
        height=400,
        title='spectrum',
        x_axis_label='freq [MHz]',
        y_axis_label='power [dB]',
        y_range=(-160, -60),
        tools=("pan", "zoom_in", "zoom_out", "box_zoom", "wheel_zoom", "save", "reset", "help", "hover"),
        sizing_mode="stretch_width",
        tooltips="@freqs_MHz, @powers_dB"  # define a tooltip using data from the x and y columns
    )

p.line(
    x="freqs_MHz",  # use the sequence in the "x_values" column
    y="powers_dB",  # use the sequence in the "y_values" column
    source=source,  # use the ColumnDataSource as the data source
)

p.scatter(
    x="marker_MHz", 
    y="marker_dB", 
    color='red',
    source=source_marker,  # use the ColumnDataSource as the data source
)

# show the results
handle = show(p, notebook_handle=True)

In [ ]:
def find_most_interesting_thing_to_record(chan, spectrum_results, dB_threshold=-100):
    'returns None if there is nothing interesting. NOT Thread safe. Called only by what_to_work_on_next()'
    freqs = spectrum_results['freqs']
    powers = spectrum_results['powers']
    sample_rate= spectrum_results['sample_rate']
    
    freq_mask = np.ones(len(freqs), dtype=bool)  # start with all frequencies up for grabs
    
    # we'll only make test transmissions in a small frequency range, so ignore huge signals outside of that range
    freq_mask *= ((freqs>=300e6)*(freqs<=330e6) +    # 4-button remote
                  (freqs>=460e6)*(freqs<=470e6) +    # walkie talkies
                  (freqs>=795e6)*(freqs<=1005e6) +   # 915 MHz B210 and drone telemetry radios
                  (freqs>=2300e6)*(freqs<=2500e6) +  # WiFi and drone control
                  (freqs>=5700e6)*(freqs<=5900e6)    # drone/car video
                 ) 
    # now freq_mask contains 1s for interestnig frequencies 0s for uninteresting ones
    
    # mask out frequency ranges that others are already recording
    NCHANS = len(spectrum_results['what_channel_is_working_on'])
    for ch in range(NCHANS):
        (task, center_freq) = spectrum_results['what_channel_is_working_on'][ch]
        if ch!=chan and task=='record':
            freq_mask *= (freqs<center_freq-sample_rate/2)+(freqs>=center_freq+sample_rate/2) # + is "or" for bools
            
    # now freq_mask contains 1s for potentially interesting signals and 0s for others
    i = np.argmax(powers*freq_mask)
    max_power_dB = 10*np.log10(powers[i])
    center_freq = freqs[i]
    if max_power_dB >= dB_threshold:
        #print('maximum power at freq:', center_freq/1e6, 'MHz   power:', max_power_dB, 'dB' )
        spectrum_results['trigger_power_dB'][center_freq] = max_power_dB  # record the power that triggered this for plotting.
        return center_freq
    else:
        return None

In [ ]:
def what_to_work_on_next(chan, spectrum_results):
    '''
    If everyone else is recording, I should take spectra no matter what.
    Otherwise, I can look for something interesting to record.
    If I should be taking spectra or find nothing to record, take spectra.

    return either
    newjob = ('recording', center_freq)
    or
    newjob = ('spectrum', i)  where i is 0..3, the part of the spectrum I should be recording
    
    This function is NOT thread safe and should be surrounded by 
    spectrum_lock.acquire()
    and 
    spectrum_lock.release()
    '''
    # count how many OTHER channels are recording:
    others_recording = 0
    NCHANS = len(spectrum_results['what_channel_is_working_on'])
    for ch in range(NCHANS):
        (task, center_freq) = spectrum_results['what_channel_is_working_on'][ch]
        if ch!=chan and task=='recording':
            others_recording += 1
            
    if others_recording < NCHANS-1:
        # Now I could look for something interesting to record
        center_freq = find_most_interesting_thing_to_record(chan, spectrum_results)
        if center_freq is not None:
            newjob = ('record', center_freq)
            spectrum_results['what_channel_is_working_on'][chan] = newjob
            return newjob
            
    # if I made it here, I should take spectra
    i = spectrum_results['next_spectral_range_index']
    spectrum_results['next_spectral_range_index'] = (i+1)%NCHANS  # let the next thread take the next range
    newjob = ('spectrum', i)
    spectrum_results['what_channel_is_working_on'][chan] = newjob
    return newjob

In [ ]:
def empowered_rx_thread(usrp, streamer, chan, spectrum_results, spectrum_lock, quit_event):
    
    # For the setup and printing, let's get the lock.
    spectrum_lock.acquire()
    
    #print('starting empowered_rx_thread')
    #print(f'starting empowered_rx_thread usrp.get_rx_rate(chan={chan}):', usrp.get_rx_rate(chan=chan)/1e6, 'MHz')
    #print('   spectrum_results.keys()', spectrum_results.keys())
    
    # update some global info that shouldn't change...
    sample_rate = spectrum_results['sample_rate']
    gain = spectrum_results['gain']  # right now, used only for recording filename
    NFFT = spectrum_results['NFFT']  # sets number of spectral bins per tuning
    center_freqs = spectrum_results['center_freqs']
    #spectrum_lock.release()  # release here if you don't need to print any more.
    
    # setup some one-time constants and buffers that will get reused
    #spectrum_Nchunks = 16  # More chunks for more accurate spectra (more averaging). Bin size is set by NFFT.
    spectrum_Nchunks = 4  # More chunks for more accurate spectra (more averaging). Bin size is set by NFFT.  For bigger MTU
    spectrum_num_samps = spectrum_Nchunks * streamer.get_max_num_samps()  # maybe it's nice to have a round number of max_num_samps
    spectrum_recv_buffer = np.zeros((1, spectrum_num_samps), dtype=np.complex64)
    #print(f'         empowered_rx_thread streamer.get_max_num_samps() for chan{chan}:', streamer.get_max_num_samps())
    print(f'         empowered_rx_thread spectrum_num_samps for chan{chan}:', spectrum_num_samps, 
                     'or', spectrum_num_samps/sample_rate, 's')
    spectrum_lock.release()  # TODO: remove and uncomment above
    
    record_time_seconds = 2.0
    record_num_samps = int(np.ceil(record_time_seconds * sample_rate))
    record_recv_buffer = np.zeros((1, record_num_samps), dtype=np.complex64)
    
    metadata = uhd.types.RXMetadata()
    
    while not quit_event.is_set():
        spectrum_lock.acquire()
        (task,value) = what_to_work_on_next(chan, spectrum_results)  # gives us a task, but also changes our status
        #print(f'   empowered_rx_thread usrp.get_rx_rate(chan={chan}):', task, value)
        spectrum_lock.release()
        
        if task=='spectrum':
            # value is now 0 for the first 1/4 of the center_freqs, 1 for the second 1/4 etc.
            # Use this value to calculate my_center_freqs
            NCHANS = usrp.get_rx_num_channels()
            NFREQS = len(center_freqs)//NCHANS  # number of center frequencies in each range
            my_center_freqs = center_freqs[value*NFREQS:(value+1)*NFREQS]
            
            for i,center_freq in enumerate(my_center_freqs):
                usrp.set_rx_freq(uhd.libpyuhd.types.tune_request(center_freq), chan=chan)
                while not usrp.get_rx_sensor('lo_locked', chan).value:
                    # Hold on, this is literally never unlocked for X310! No need to even check and wait.
                    time.sleep(0.0001)
                    not_locked_count += 1
                # Wait just for things to settle, but not long for X310. Longer seems to be cleaner.
                #time.sleep(0.0001)  # This gives crazy shit in first bin for X310 TwinRX
                #time.sleep(0.001)  # Even 1ms slows things down but not long enough to settle
                time.sleep(0.002)  # 2ms is the shortest time that gives a clean 915 MHz signal. Not sure about the rest.
                
                # Start Spectrum Stream
                stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.num_done) # stream num_samps and then done
                #stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.start_cont) # stream continuous, so we can start recording right away
                stream_cmd.stream_now = True
                stream_cmd.num_samps = spectrum_num_samps
                streamer.issue_stream_cmd(stream_cmd)  # often just stream num_samps
                # Receive Samples
                streamer.recv(spectrum_recv_buffer, metadata)  # writes into metadata
                #if i==0:
                #    print('chan', chan, 'center_freq', center_freq, 'spectrum_recv_buffer:', spectrum_recv_buffer[-4:])
                # TODO: Also save metadata info
                
                #recv_stack[i,:] = recv_buffer[0]  # record these samples for this center frequency (slow for debug but maybe we save the trigger)
                valid_buffer = spectrum_recv_buffer[len(spectrum_recv_buffer)//2,:]  # used to take second half, now we just wait longer
                #valid_buffer = spectrum_recv_buffer[0,:]  # used to take second half, now we just wait longer
                
                # calculate PSD on these samples with a given bin spacing
                # WARNING: You need to copy the buffer, as we do into valid_buffer. Otherwise PSD gives zero. I don't know why. A threading or shared memory issue.
                psd_power, psd_frequencies = matplotlib.mlab.psd(
                    valid_buffer, NFFT=NFFT, Fs=sample_rate, 
                    window=matplotlib.mlab.window_hanning, # window_none turns real spectral lines into wide volcanos due to discontinuity
                    detrend=matplotlib.mlab.detrend_mean)  # detrend_none makes huge DC spike. detrend_linear might help or not but can't change after calib
                #print('psd:', len(psd_power), len(psd_frequencies))
                #assert(len(psd_power)==NFFT and len(psd_frequencies)==NFFT)  # True for complex samples
                CHEAT = True
                if CHEAT:
                    # Cheat on the DC 0 offset bin by replacing the center-frequency with an average of the ones next to it
                    psd_power[NFFT//2]  = (psd_power[NFFT//2-1] + psd_power[NFFT//2+1])/2.0  # CHEAT!
                    # Cheat on the ends by replacing them with their neighbors
                    psd_power[0] = psd_power[2]  # CHEAT!
                    psd_power[1] = psd_power[2]  # CHEAT!
                    psd_power[NFFT-1] = psd_power[NFFT-2]  # CHEAT!
                
                # Now copy the frequencies and powers into the long power spectrum around this particular center frequency
                f = value*NFREQS + i # the starting NFFT-length chunk number
                spectrum_lock.acquire()
                #print('Writing freqs and powers chan', chan, 'center_freq', center_freq, 'psd_power[0]', psd_power[0])
                spectrum_results['freqs'][f*NFFT:(f+1)*NFFT] = psd_frequencies + center_freq
                spectrum_results['powers'][f*NFFT:(f+1)*NFFT] = psd_power
                #is_interesting = this_is_interesting_and_we_should_record()
                spectrum_lock.release()
                #if is_interesting:
                #    pass
                #else:
                #    # Stop Stream
                #    stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.stop_cont)
                #    streamer.issue_stream_cmd(stream_cmd)
                
        if task=='record':
            # we might reach here from the top or because a spectral bin was interesting enough to keep recording
            center_freq = value
            
            # set up stream command.
            stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.num_done) # stream num_samps and then done
            #stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.start_cont) # stream continuous
            stream_cmd.stream_now = True
            stream_cmd.num_samps = record_num_samps
            
            usrp.set_rx_freq(uhd.libpyuhd.types.tune_request(center_freq), chan=chan)
            while not usrp.get_rx_sensor('lo_locked', chan).value:
                # Maybe do processing from the previous step before or while we wait
                time.sleep(0.01)
            
            time.sleep(0.02)  # Wait just for things to settle, but not long for X310
            #time.sleep(0.10)  # really long time to test laptop issues
            # Start Stream
            streamer.issue_stream_cmd(stream_cmd)  # start the stream
            # record time for the filename or SigMF data
            start_time = datetime.datetime.now(datetime.timezone.utc)  # this is from the computer
            # TODO: look at metadata that gets filled in below for a possibly better timestamp from the USRP
            # Receive Samples
            streamer.recv(record_recv_buffer, metadata)
            #print(f'rx_thread_record chan={chan}  recv_buffer:', recv_buffer[-5:])
            #print(f'rx_thread_record chan={chan}  metadata:', metadata)
            # TODO: check if power is still there and we should still be recording
            '''
            # Stop continuous streaming if that's what we're doing
            stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.stop_cont)
            streamer.issue_stream_cmd(stream_cmd)
            '''
            # Currently inspectrum can only read files with interleaved (complex) 32-bit floats,
            timestring = start_time.strftime("%Y%m%d_%H%M%SZ")  # UTC time, so mark it with a Z
            output_file = f"record_{timestring}_chan_{chan}_center_freq_{center_freq}_gain_{gain}_sample_rate_{sample_rate}_complex64.bin"
            with open(output_file, 'wb') as out_file:
                if False: #args.numpy:
                    np.save(out_file, record_recv_buffer, allow_pickle=False, fix_imports=False)
                else:
                    record_recv_buffer.tofile(out_file)
        #time.sleep(0.5) # just to slow things down for debug
    # we got a quit signal
    print('ending empowered_rx_thread')

In [ ]:
usrp = uhd.usrp.MultiUSRP("type=x300")  # let driver pick size automatically. If MTU and buffer size are NOT set properly, it will suggest things.
#usrp = uhd.usrp.MultiUSRP("type=x300,recv_frame_size=9000") # UHD recommends a send frame size of at least 8000 for best performance
#usrp = uhd.usrp.MultiUSRP("type=x300,num_recv_frames=1000")  # if you are trying to receive at a high rate but are getting overflows, make the receive buffer much larger (the default value is 32)

# TODO: Lock time to GPSDO 
# See https://files.ettus.com/manual/page_gpsdo_x3x0.html
# and example https://github.com/EttusResearch/uhd/blob/master/host/examples/sync_to_gps.cpp
# try running /usr/lib/uhd/examples/sync_to_gps --args "type=x300"

#sample_rate = 100e6 # Hz max X310 but filter rolloff
sample_rate = 50e6 # Hz for X310 with less rolloff
gain = 50 # dB for X310 TwinRX
#gain = 99 # dB max for X310 TwinRX

# Make the list of center_freqs for the big global spectrum
lo_freq_range = usrp.get_rx_freq_range()  # Check what our radio is capable of
if lo_freq_range.start() > sample_rate/2:  # lowest RX frequency is significantly above zero, Like the B210
    center_freqs = np.arange(lo_freq_range.start(), lo_freq_range.stop()+sample_rate, sample_rate)
else:  # lowest RX frequency is close to zero. Like X310
    #center_freqs = np.arange(sample_rate/2, lo_freq_range.stop(), sample_rate)
    center_freqs = np.arange(sample_rate, lo_freq_range.stop(), sample_rate)  # start higher, offset by sample_rate/2
    #center_freqs = np.arange(sample_rate/2*3/2, lo_freq_range.stop(), sample_rate)  # offset by a half-spacing. ends ok now
print('Number of center_freqs:', len(center_freqs), 'for', 
          lo_freq_range.start()/1e6, 'MHz to', 
          lo_freq_range.stop()/1e6, 'MHz at',
          sample_rate/1e6, 'MSPS')

# parameters for the big global spectrum
NFFT = 256 # original testing here
#NFFT = 1024
freqs = np.zeros(len(center_freqs)*NFFT)  # frequencies for each small spectral bin of large spectrum. TODO: if there is overlap, change this
spectrum_results = {  # access this dictionary ONLY between spectrum_lock.acquire() and spectrum_lock.release()
    # Some global shared info that shouldn't change
    'sample_rate'      : sample_rate,
    'gain'             : gain, # maybe this can be changed per-channel based on signal strength
    'NFFT'             : NFFT,
    'center_freqs'     : center_freqs,
    'freqs'            : freqs,
    # spectrum threads update this:
    'powers'           : np.zeros(len(freqs))+1e-200,  # Before we go take data, be sure not to find any interesting power
    'psd_time'         : np.zeros(len(freqs)),  # how long did it take to record these channels in unix timestamp
    'timestamps'       : np.zeros(len(freqs)),  # times when each spectral bin was last recorded
    'by_chan'          : np.zeros(len(freqs)),  # which channel recorded each spectral bin
    # things for the threads to coordinate their next action:
    'what_channel_is_working_on' : [('idle',None)]*usrp.get_rx_num_channels(),  # list of ('task', value) pairs. task is 'record' or 'spectrum'
    'next_spectral_range_index'  : 0,  # goes 0...NUM_CHANNELS-1.  If a channel should take a spectrum, do this piece.
    'trigger_power_dB' : {},  # a map from trigger frequency to trigger power. For plotting.
}
#recording_results = {}  # use this without locks, so don't access it from the main thread until the worker has joined

# threads inspired by the benchmark_rate.py example...

threads = []
quit_event = threading.Event()  # a signal for the threads to stop running
spectrum_lock = threading.Lock() # a lock to share spectrum data


# For now, set all 4 channels to the same parameters
for chan in range(usrp.get_rx_num_channels()):
    usrp.set_rx_rate(sample_rate, chan=chan)
    usrp.set_rx_gain(gain, chan=chan)
    st_args = uhd.usrp.StreamArgs("fc32", "sc16")  # complex float32 in CPU, signed int16 over the wire
    st_args.channels = [chan]
    streamer = usrp.get_rx_stream(st_args)
    thread = threading.Thread(target=empowered_rx_thread,
                  args=(usrp, streamer, chan, spectrum_results, spectrum_lock, quit_event))
    thread.name = f"thread_chan{chan}"
    thread.start()
    threads.append(thread)

while np.any([t.is_alive() for t in threads]):  # while any thread is still alive
    spectrum_lock.acquire()
    # Update the Bokeh spectrum plot above. 
    # Math is done here, so we don't have to worry about the data changing after we release the lock
    source.data = {
        "freqs_MHz" : spectrum_results['freqs']/1e6, # x
        "powers_dB" : 10*np.log10(spectrum_results['powers']), # y
    }
    marker_freq = np.array([
                center_freq for (task, center_freq) 
                       in spectrum_results['what_channel_is_working_on'] 
                           if task=='record'])
    marker_power = [spectrum_results['trigger_power_dB'][freq] for freq in marker_freq]
    source_marker.data = {
        "marker_MHz" : marker_freq/1e6, # x
        "marker_dB"  : marker_power, # y
    }
    spectrum_lock.release()
    push_notebook(handle=handle)
    time.sleep(1/15) # for a rough number of frames per second FPS

# Plot a recorded signal

In [ ]:
!ls record*

In [ ]:
filename = 'record_20240406_045943Z_chan_0_center_freq_2464257812.5_gain_50_sample_rate_50000000.0_complex64.bin'
a = np.fromfile(filename, dtype=np.complex64)
a = a[:200*1000]  # take the first 4ms

In [ ]:
p = figure(
        height=400,
        title='amplitude',
        x_axis_label='time [ms]',
        y_axis_label='amplitude',
        tools=("pan", "zoom_in", "zoom_out", "box_zoom", "wheel_zoom", "save", "reset", "help", "hover"),
        sizing_mode="stretch_width",
    )
time = np.arange(len(a))/sample_rate*1e3 # ms
p.line(x=time, y=a.real, alpha=0.5)
p.line(x=time, y=a.imag, alpha=0.3, color='red')
show(p)

### TODO

* Record the time that a bin crossed the threshold and the time streaming started. Laptop time and UHD time
* SigMF metadata with int16 unfiltered output recording. float32 for inspectrum. 
* A list of data recording plots that can be added to. Demod the button or Walkie Talkie?
* Message passing instead of shared dictionary to allow many X310 boxes on different computers?
* Stream continuously when recording and have a stop criteria.
* What happens if we are taking spectra and get a bright chunk?
    - Go onto the next chunk of spectra or switch to recording mode, saving the data that triggered this?
    - Maybe just keep recording in that channel and stop somewhere else if it's lower priority. Continuous vs N samples?
    - Check if any samples are lost when switching from streaming num_done (N samples and then stop) to start_cont (continuous)
* Conversely, while it's recording, it could be contributing to the global spectrum. You have to start centered on a spectral bin.
* Demo with key fob buttons, walkie talkie, RFM69, Lora burst, Drone TX, B210 ZC random
* Maybe spectrum takes Pseudorandom hops to get spectral chunks at the cost of PLL lock time?
* RFNoC FFT for X310?
* Can you continuously stream on one channel in one thread while changing carrier and toggling streaming with the other? Yes. Sample rate? Not sure.
* Special channels connected to different antennas?
* Gain for each channel?
* Gain set for recording based on signal strength?
* Overlap the taking of spectra to not cheat in the middle and edge bins
* Calibrate spectra to physics dBm or at least flat across each sub-band